## Introduction to IPKISS Waveguides

To introduce you to IPKISS waveguide concept, let's first take a look at some waveguide examples from our **PICAZZO** library (PICAZZO is a generic component library with a wide range of photonic components in IPKISS).

Before that, we import the necessary libraries as usual, including the IPKISS module, the *silicon_photonics* PDK (a PDK for demonstration purpose), and for plotting. 

<div class="panel panel-info">
 <div class="panel-heading">New to Python?</div>
 <div class="panel-body">
    <p>If you haven't heard about Python before, don't worry, it's easy to learn. You should be able to follow the waveguides tutorial without being a Python expert. If you want to learn more about Python concepts, we also included a standard Python tutorial [Intro to Python](../python_getting_started/Basic Introduction to Python.ipynb).</p>
  </div>  
</div>


In [ ]:
%matplotlib inline
from technologies import silicon_photonics
from ipkiss3 import all as i3
import pylab as plt
import numpy as np
import sys

plt.rcParams['figure.figsize'] = (8, 8)
sys.path.insert(0, '.')

In IPKISS, we draw waveguide using **trace template** and  waveguide class such as **RoundedWaveguide**

A trace template defines all the aspects of the waveguide that could be extracted from its cross-section:

- how the waveguide is draw along the shape of the waveguide,
- the simulation model, e.g. effective index and group index of the waveguide,
- and the geometrical cross-section, can be derived from the layout through virtual fabrication.

The waveguide class determines where is drawn 

- Path of the waveguide
- Bending radius
- ...


For this reason, many parametric cells in ipkiss take one or multiple `trace_template` parameters, which determines the template(s) of the waveguides that are part of that pcell. This is best illustrated by the image below.


Along with the trace, we obtain the actual layout.
<img src="_images/ring.png" width='60%'</a>

Lets now try how this works on regular a simple waveguide, in order to build a waveguide bend:

In [ ]:
# Trace Template 
from picazzo3.traces.wire_wg.trace import WireWaveguideTemplate # Predefined Trace template from Picazzo


wg_t = WireWaveguideTemplate()
wg_t.Layout(core_width=0.47,
            cladding_width=2 * 3.0 + 0.47,
            core_process=i3.TECH.PROCESS.WG)


wg2_t = WireWaveguideTemplate()
wg2_t.Layout(core_width=1.47,
            cladding_width=2 * 3.0 + 0.47,
            core_process=i3.TECH.PROCESS.WG)


# Waveguide

wg = i3.RoundedWaveguide(trace_template=wg_t)
wg_l = wg.Layout(shape=[(0.0, 0.0), (10.0,0),(10.0,10.0), (50.0,50.0)], 
                 bend_radius=5.0)
wg_l.visualize()
wg_l.write_gdsii("test.gds")

We can also visualize the cross-section of the waveguide as follows:

In [ ]:
# Visualize a cross section

wg_l.cross_section(cross_section_path=i3.Shape([(2.0, -5.0), (2.0, 5.0)])).visualize()

The above example showed how to draw a simple piece of waveguide with trace template. Next, we will show some relevant examples on how to draw waveguides.


## Using Routes


Often you do not want to specify all the waypoints inside a waveguide yourself, but you just care about the start and and port. This is a good case to use routing functions. For example, a route created by `RouteManhattan` function (\*)  creates Manhattan-like (orthogonal) wire between two define ports.

(\*) In our documentation, you may find various types of routing functions: <a href="http://docs.lucedaphotonics.com/3.1.0/reference/layout/routing.html"> Routing functions</a>

In [ ]:
input_port = i3.OpticalPort(name="in", position=(5.0, 0.0), angle_deg=20.0, trace_template=wg_t)
output_port = i3.OpticalPort(name="out", position=(50.0, 30.0), angle_deg=180.0, trace_template=wg_t)

# create the route object
route = i3.RouteManhattan(input_port=input_port,
                          output_port=output_port)

# a route is a Shape, so we can use it to draw a waveguide
wg = i3.RoundedWaveguide(trace_template=wg_t)
layout = wg.Layout(shape=route)
layout.visualize()

## Ports

Note that we also introduced the concept of an ``OpticalPort`` here. An optical port has a name, position and angle. It also contains the **trace template**.

In [ ]:
input_port.trace_template

### Define a custom waveguide template

In addition to waveguide templates you may find in PICAZZO library, you can also define your own waveguide template using `WindowWaveguideTemplate`:

In [ ]:
from ipkiss3.pcell.photonics.waveguide import WindowWaveguideTemplate
from ipkiss3.pcell.trace.window.window import PathTraceWindow

class MyWgTemplate(WindowWaveguideTemplate):
    
    class Layout(WindowWaveguideTemplate.Layout):
        core_width = i3.PositiveNumberProperty(doc="Core width of the waveguide")
        cladding_width = i3.PositiveNumberProperty(doc="Cladding width of the waveguide")
        
        def _default_core_width(self):
            return 0.45
        
        def _default_cladding_width(self):
            return 4.0
        
        def _default_cover_layers(self):
            # Layer for Manhattan rectangles (drawn in waveguide bends when the waveguide manhattan parameter = True)
            return []

        def _default_windows(self):
            windows = []
            windows.append(PathTraceWindow(layer=i3.TECH.PPLAYER.WG.CORE,
                                           start_offset=-0.5 * self.core_width,
                                           end_offset=+0.5 * self.core_width))

            windows.append(PathTraceWindow(layer=i3.TECH.PPLAYER.WG.CLADDING,
                                           start_offset=-0.5 * self.cladding_width,
                                           end_offset=+0.5 * self.cladding_width))
            return windows

Now, we use this created waveguide template to draw a simple shape:

In [ ]:
# Instantiate the new waveguide template to use it in our waveguide below
wg_tmpl = MyWgTemplate()

wg = i3.RoundedWaveguide(trace_template=wg_tmpl)
wg_lay = wg.Layout(shape=[(0, 0), (5, 0), (11, 5)])

wg_lay.visualize()

Congratulations! You have now been introduced to the IPKISS waveguide concept, which is useful for drawing almost any photonic components and routing the photonic circuit. 